# 🍸 Review Crawling 
---
- 3535개의 위스키에 대한 약 90만건의 review data 존재합니다.
- 이를 크롤링하기 위한 코드입니다.

## 필요 라이브러리
- bs4
- requests
- time
- pandas
- tqdm
- re
- json

In [1]:
from bs4 import BeautifulSoup
import requests

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import os
import time as tt
import pandas as pd
from tqdm.notebook import tqdm
import re

import json
import math

### whisy data 불러오기

In [2]:
path = os.getcwd()
whiskies = pd.read_csv(path + "\\dataset\\whisky.csv", index_col = 0)
whiskies

,link,image,name,avr_rating,category,location,total_rating,cost_rank,abv,cask_type,...,oily,full_bodied,rich,sweet,briny,salty,vanilla,tart,fruity,floral
0,/spirits/hibiki-21-year,https://ip-distiller.imgix.net/images/spirits/...,Hibiki 21 Year,4.52,Blended,Japan,861,5,43.00,"ex-bourbon American oak, ex-sherry European oa...",...,20,80,80,85,0,15,20,25,85,50
1,/spirits/highland-park-18,https://ip-distiller.imgix.net/images/spirits/...,Highland Park 18 Year,4.47,Peated Single Malt,"Islands, Scotland",2987,4,43.00,ex-sherry,...,40,70,80,70,20,40,50,50,70,20
2,/spirits/michter-s-20-year-kentucky-straight-b...,https://ip-distiller.imgix.net/images/spirits/...,Michter's 20 Year Kentucky Straight Bourbon (2...,4.61,Bourbon,"Kentucky, USA",10,5,57.10,"new, charred American oak",...,15,80,90,85,0,5,30,25,35,0
3,/spirits/george-t-stagg-bourbon-fall-2019,https://ip-distiller.imgix.net/images/spirits/...,George T. Stagg Bourbon (Fall 2019),4.61,Bourbon,"Kentucky, USA",628,4,58.45,"new, charred American oak",...,20,60,60,45,0,0,60,60,45,0
4,/spirits/bowmore-mizunara-cask-finish,https://ip-distiller.imgix.net/images/spirits/...,Bowmore Mizunara Cask Finish,4.08,Peated Single Malt,"Islay, Scotland",22,5,53.90,"ex-bourbon, ex-sherry, new Mizunara oak",...,10,75,75,60,15,20,30,20,50,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3530,/spirits/wild-turkey-spiced,https://ip-distiller.imgix.net/images/spirits/...,Wild Turkey Spiced,3.00,Flavored Whiskey,"Kentucky, USA",13,1,43.00,"New, Charred American Oak",...,0,40,55,85,0,0,25,10,10,0
3531,/spirits/seagram-s-seven-crown-american-blende...,https://ip-distiller.imgix.net/images/spirits/...,Seagram's 7 Crown American Blended Whiskey,2.21,Blended American Whiskey,USA,237,1,40.00,oak,...,0,30,0,100,0,0,100,0,60,10
3532,/spirits/11-wells-single-malt-whiskey,https://ip-distiller.imgix.net/images/spirits/...,11 Wells Single Malt Whiskey,3.00,American Single Malt,"Minnesota, USA",2,3,42.00,"new, charred American oak",...,40,10,10,90,0,0,50,20,30,10
3533,/spirits/immortal-spirits-early-whiskey,https://ip-distiller.imgix.net/images/spirits/...,Immortal Spirits Early Whiskey,1.00,Other Whiskey,"Oregon, USA",1,2,44.50,"new, charred American oak",...,80,0,0,40,0,0,20,0,0,0


## 크롤링은 할당량

- 다운 : [0:550]
- 창민 : [550:1100]
- 지은 : [1100:1650]
- 은성 : [1650:2200]
- 예진 : [2200:2750]
- 준현 : [2750:3535]

### ✨ start와 end 값만 수정하시면 됩니다!!

In [6]:
start = 3400
end = 3535

### link 기반 크롤링 -> BeautifulSoup 사용

https://distiller.com/spirits/william-larue-weller-bourbon-fall-2016/tastes?page=1

In [7]:
whisky_names = []
usernames = []
ratings = []
for i, whisky in tqdm(whiskies[start:end].iterrows(), desc="outer", position=0):
    whisky_name = whisky["name"]
    
    base_url = "https://distiller.com" + whisky["link"] + "/tastes?page="
    last_page = math.ceil(whisky["total_rating"] / 10)
    print("====================================================================================================================================")
    print("index : {} total_rating : {} totalPage : {}".format(i, whisky["total_rating"], last_page))
    
    for page in tqdm(range(1, last_page + 1), desc="inner", position=1, leave=True):
        url = base_url + str(page)
        req = requests.get(url)
        print("url : {}".format(url))
        tt.sleep(1.5)
        soup = BeautifulSoup(req.text, "lxml")
        
        rating_details = soup.find_all(class_="name-details")
        for info in rating_details:
            try: 
                username = info.select_one("h3.mini-headline").get_text().strip()
            except AttributeError: 
                username = "Unknown"
            
            try:
                rating = math.floor(float(info.select_one("div.rating-display__value").get_text()) * 2)
            except AttributeError:
                rating = None
            
            whisky_names.append(whisky_name)
            usernames.append(username)
            ratings.append(rating)

outer: 0it [00:00, ?it/s]

index : 3400 total_rating : 68 totalPage : 7


inner:   0%|          | 0/7 [00:00<?, ?it/s]

url : https://distiller.com/spirits/yellow-rose-blended-whiskey/tastes?page=1
url : https://distiller.com/spirits/yellow-rose-blended-whiskey/tastes?page=2
url : https://distiller.com/spirits/yellow-rose-blended-whiskey/tastes?page=3
url : https://distiller.com/spirits/yellow-rose-blended-whiskey/tastes?page=4
url : https://distiller.com/spirits/yellow-rose-blended-whiskey/tastes?page=5
url : https://distiller.com/spirits/yellow-rose-blended-whiskey/tastes?page=6
url : https://distiller.com/spirits/yellow-rose-blended-whiskey/tastes?page=7
index : 3401 total_rating : 194 totalPage : 20


inner:   0%|          | 0/20 [00:00<?, ?it/s]

...

url : https://distiller.com/spirits/11-wells-single-malt-whiskey/tastes?page=1
index : 3533 total_rating : 1 totalPage : 1


inner:   0%|          | 0/1 [00:00<?, ?it/s]

url : https://distiller.com/spirits/immortal-spirits-early-whiskey/tastes?page=1
index : 3534 total_rating : 16 totalPage : 2


inner:   0%|          | 0/2 [00:00<?, ?it/s]

url : https://distiller.com/spirits/triple-crown-blended-whiskey/tastes?page=1
url : https://distiller.com/spirits/triple-crown-blended-whiskey/tastes?page=2


In [8]:
print(len(whisky_names))
print(len(usernames))
print(len(ratings))

11259
11259
11259


In [9]:
savePath = os.getcwd()+"\\dataset\\rating"
savePath

'C:\\Users\\SSAFY\\Desktop\\Whisky\\dataset'

(현재 작업 위치)\\dataset 위치에 생성한 데이터를 csv 파일로 저장합니다.

### 파일 저장
- start, end 기준으로 파일명을 저장합니다.

In [10]:
df = pd.DataFrame({
    "username": usernames, "whisky_name": whisky_names, "rating": ratings 
})
df.to_csv(savePath+"\\user_whisky_rating_{}_{}.csv".format(str(start), str(end)))